In [1]:
!export HF_HOME="/scratch/ssd004/scratch/lfy"

In [2]:
from llama_cpp import Llama, LlamaGrammar
import json

In [3]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [52]:
llm = Llama.from_pretrained(
    repo_id="QuantFactory/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="*Q4_K_M.gguf",
    # n_gpu_layers=-1,
    n_ctx=128000,
    verbose=True,
    cache_dir="/checkpoint/lfy/13870962"
)

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /checkpoint/lfy/13870962/models--QuantFactory--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/b6d5cca03f341fd97b7657420bd60e070835b7e5/./Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam

In [53]:
ask_food_prompt_template = "The user is living in {user_location}. The user is looking for a {user_query}. Suggest 5 {user_query} which can be cooked by the user, without actual recipe."

In [47]:
user_location = "California"
user_query = "vegan korean food"
ask_food_prompt = ask_food_prompt_template.format(user_location=user_location, user_query=user_query)

In [48]:
schema_food_name = r'''
root ::= (
    "{" newline
        doublespace "\"food_names\":" space listofstring newline
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
listofstring ::= ("[" space (string ("," space string){4})? "]")
'''

# Creating a LlamaGrammar object with schema string
# Set verbose=False to not print the grammar, set to True for debugging
grammar_food_name = LlamaGrammar.from_string(grammar=schema_food_name, verbose=False)

In [49]:
result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON.",
        },
        {"role": "user", "content": ask_food_prompt},
    ],
    grammar=grammar_food_name,
    temperature=0.7,
)

llama_perf_context_print:        load time =     904.89 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    61 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    45 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5801.66 ms /   106 tokens


In [9]:
result_json = json.loads(result['choices'][0]['message']['content'])
result_json

{'food_names': ['Vegan Bibimbap',
  'Kimchi Stew',
  'Korean-Style Vegan Tacos',
  'Vegan Japchae',
  'Doenjang Jjigae']}

In [10]:
query_res = [{"index":0,"query":"kimichi stew","Name":"Perfect Chicken Stew","RecipeInstructions":"c(\"Season a 3-7 pound chicken with Garlic powder and Pepper. Roast chicken in oven at 325 degrees.\", \"While chicken is cooking, dice potatoes, slice carrots, chop onions and carrots to desired thickness. Place vegetables in stewing pot and add water until vegetables are covered with about an 3 inches of water. Boil rapidly until potatoes are just finished.\", \"Remove vegetables from the pot by straining them and keep the water. By removing the vegetables and letting them cool, you prevent overcooking them and they won't dissolve into nothing.\", \n\"With remaining water on low heat, add can of cream of mushroom soup, can of chicken stock and milk (milk optional, Zie Ga Zink).\", \"If you don't use milk, I suggest a premium ready to serve brand of creamed mushroom soup, it will be of a smoother, creamier consistency than the regular cans of mushroom soup.\", \"Get a small sealable container and fill with 1 cup of cold water, then add 1 cup of flour, cover and seal, then immediately shake vigorously. You are making a thickener for the stew, it should look like the consistency of glue with no lumps. If to thick add a bit of water, too thin add a bit more flour, shake very hard again. If there are a few lumps you can remove them by straining. This process, once learned, is very useful for making gravies or other stews without using a high-fat butter and flour 'roux' thickener.\", \n\"Rapidly add thickener to the starch water/mushroom soup/stock/milk mixture using a whisk. You may have to make a little more thickener if you want a hardier stew, just remember that the stew will thicken more after it is removed from the heat and it stands. Simmer to desired consistency. Stir often. Do not burn! I suggest a non-stick stew pot, it helps prevent burning.\", \"Add the cooked (now cooled) vegetables to the stew.\", \"When chicken is finished roasting, drain juices into the stew. Remove skin and bones.  Tear or cut chicken apart and add to the stew.\", \n\"Stir in about 2-3 tablespoons of salt to stew  and about the same amount of pepper to taste.\", \"If you want, try adding a dash of hot sauce or a pinch of Sambel Olek.\", \"Let stew simmer for a little longer. Serve with fresh bread and Enjoy.\", \"Questions? brennarlauterbach@hotmail.com.\")"},{"index":1,"query":"kimichi stew","Name":"Wintry Beef Vegetable Stew With Fluffy Herb Dumplings","RecipeInstructions":"c(\"Cook and stir beef in shortening in heavy 8-10 quart stock pot, until beef is well browned. (Note: If too much liquid builds up to prevent adequate browning, pour off excess liquid into a bowl and reserve. Continue to brown the beef and when well browned, add the reserved liquid back into the pot.).\", \"Add 5 cups hot water, 1/2 teaspoon salt and the black pepper.\", \"Heat to boiling; reduce heat.\", \"Cover and simmer until beef is almost tender, 45 minutes to 1 hour.\", \"Stir in potato, turnip, rutabaga, carrots, green pepper, green beans (if using), celery, onion, bouquet sauce, the bouillon cube and bay leaves.\", \n\"Cover and simmer until vegetables are tender (but do not overcook), stirring once, about 25 minutes.\", \"Prepare dough (see below) for Dumplings;  set aside.\", \"Using a fork, blend together 1 cup cold water and the 4 tablespoons flour in a small mixing bowl; stir gradually into stew.\", \"Heat to boiling, stirring constantly.\", \"Boil and stir 1 minute; reduce heat.\", \"Do ahead tip: After boiling and stirring 1 minute, stew can be covered and refrigerated no longer than 48 hours. To serve, heat to boiling over medium-high heat. Continue as directed.\", \n\"DUMPLINGS:\", \"In a large bowl, cut shortening into combined flour, baking powder, salt, parsley and herbs until mixture resembles fine crumbs.\", \"Stir in milk.\", \"Drop by heaping tablespoons onto hot meat or vegetables in boiling stew (do not drop directly into liquid).\", \"Cook uncovered 15 minutes.\", \"Cover and cook about 15 minutes longer. Cut a dumpling in half to test for doneness; you want them done but not dry!\", \"Serve stew piping hot, with a buttered baguette and a glass of cider, ale, or wine. As with all good stews, this stew is even better reheated the next day, after flavors have had a chance to meld. Stew leftovers freeze and reheat beautifully, and would make a delicious cottage or shepherd's pie.\"\n)"}]

In [11]:
import re
extracted_recipe = {}
for recipe in query_res:
    recipe_name = recipe['Name']
    recipe_instructions = recipe["RecipeInstructions"]
    matches = re.findall(r'"(.*?)"', recipe_instructions)
    recipe_instructions = tuple(matches)
    steps_dict = {f"Step {i+1}": step for i, step in enumerate(recipe_instructions)}
    steps_json = json.dumps(steps_dict, indent=2)
    extracted_recipe[recipe_name] = {} # Overwrite recipe with same name, no duplicate
    extracted_recipe[recipe_name]['instructions'] = steps_json

In [12]:
ask_recipe_template = "The user is living in {user_location}. The user is looking for a {user_query}. Here are few potential recipes: {retrieved_recipes}. Based on such recipe, generate a new recipe that satisfy user nutrition requirements. You can reuse the existing recipe, or you can modify it or create a new recipe."

In [13]:
ask_recipe_prompt = ask_recipe_template.format(user_location=user_location, user_query=user_query, retrieved_recipes=extracted_recipe)

In [54]:
# schema_recipe = r'''
# root ::= (
#     "{" newline
#         doublespace "\"Description\":" space string "," newline
#         doublespace "\"Cooking instructions\":" cookinstructs "," space 
#         # doublespace "\"Ingredients\":" space ingreds newline
#     "}"
# )
# newline ::= "\n"
# doublespace ::= "  "
# number ::= [0-9]+   "."?   [0-9]*
# integer ::= [0-9]*
# boolean ::= "true" | "false"
# char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
# space ::= | " " | "\n" [ \t]{0,20}
# string ::= "\"" char* "\"" space
# sentence ::= char* space
# listofstring ::= ("[" space (string ("," space string)*)? "]")
# cookstep ::= ("\"Step" space integer "\"" ":" space string)
# cookinstructs ::= (space "{" space (cookstep ("," space cookstep){20})? "}")
# ingred ::= ("{" space "\"Ingredient\":" space string ", \"Unit\": " string ", \"Amount\": " number "}")
# ingreds ::= (space "\[" space (ingred ("," space ingred){0,30}) "\]")
# '''

schema_recipe = r'''
root ::= (
    "{" newline
        doublespace "\"Description\":" space string "," newline
        doublespace "\"Cooking instructions\":" cookinstructs "," space 
        doublespace "\"Ingredients\":" space listofstring newline
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
integer ::= [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
sentence ::= char* space
listofstring ::= ("[" space (string ("," space string)*)? "]")
cookstep ::= ("\"Step" space integer "\"" ":" space string)
cookinstructs ::= (space "{" space (cookstep ("," space cookstep){10})? "}")
'''

grammar_recipe = LlamaGrammar.from_string(grammar=schema_recipe, verbose=False)

In [55]:
result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON. You must not include nutrition information or any notes. Stop generating instructions when the food is ready to serve. Do not put notes or nutrition in cooking instructions.",
        },
        {"role": "user", "content": ask_recipe_prompt},
    ],
    grammar=grammar_recipe,
    temperature=0.7,
    stream=True
)

for chunk in result:
    delta = chunk['choices'][0]['delta']
    if 'role' in delta:
        print(delta['role'], end=': ')
    elif 'content' in delta:
        print(delta['content'], end='')


assistant: {
  "Description": "Vegan Korean-Style Japchae Stew",
  "Cooking instructions": {
    "Step 1": "Soak 1 cup of dried Korean glass noodles (japchae) in water for at least 4 hours. Drain and cut into 2-inch pieces.",
    "Step 2": "In a large pot, heat 2 tablespoons of sesame oil over medium heat. Add 1 onion, diced, and cook until softened.",
    "Step 3": "Add 2 cloves of garlic, minced, and cook for 1 minute.",
    "Step 4": "Add 1 cup of mushrooms, sliced, and cook until they release their liquid and start to brown.",
    "Step 5": "Add 1 cup of diced zucchini, 1 cup of diced bell peppers, and 1 cup of diced carrots. Cook for 2-3 minutes.",
    "Step 6": "Add 2 tablespoons of Gochujang (Korean chili paste) and 2 tablespoons of soy sauce. Stir to combine.",
    "Step 7": "Add the soaked and cut glass noodles to the pot. Stir to combine with the vegetables and sauce.",
    "Step 8": "Cook for an additional 2-3 minutes, until the noodles are well coated with the sauce.",
    

llama_perf_context_print:        load time =   17960.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1376 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   481 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   75603.37 ms /  1857 tokens


}

In [ ]:
print(result['choices'][0]['message']['content'])

TypeError: 'generator' object is not subscriptable

In [ ]:
json.loads(result['choices'][0]['message']['content'])

{'Description': 'Vegan Korean-Style Vegetable Stew with Seitan and Gochujang',
 'Cooking instructions': {'Step 1': 'Cook the seitan in a pot of boiling water for 10-15 minutes, or until it is cooked through.',
  'Step 2': 'In a separate pot, heat some oil over medium heat. Add the garlic, ginger, and onions, and cook until the onions are translucent.',
  'Step 3': 'Add the gochujang, soy sauce, and sugar to the pot, and stir to combine.',
  'Step 4': 'Add the vegetables (such as zucchini, bell peppers, and carrots) to the pot, and cook for 5-7 minutes, or until they are tender.',
  'Step 5': 'Add the cooked seitan to the pot, and stir to combine.',
  'Step 6': 'Season the stew with salt and black pepper to taste.',
  'Step 7': 'Serve the stew hot, garnished with green onions and sesame seeds.',
  'Step 8': 'Note: You can customize this recipe by using different types of vegetables, or by adding other ingredients such as tofu or tempeh.',
  'Step 9': 'Nutritional information: This recip